In [1]:
import json
import pandas as pd
import os




In [2]:
def read_vuln_json(vuln_dir='data/vuln/json/', data = {}):
    for filename in os.listdir(vuln_dir):
        with open(vuln_dir+filename) as f:
            data.update(json.load(f))
    return data

def parse_cve_json(cve_data,columns=['CVE','vendor','product','version'], save_loc="data/vuln/data.pkl"):
    df = pd.DataFrame(columns=['CVE','vendor','product','version'])
    for item in cve_data["CVE_Items"]:
        CVE = item["cve"]['CVE_data_meta']['ID']
        try:
            for vendor in item['cve']['affects']['vendor']['vendor_data']:
                vendor_name = vendor['vendor_name']
                for product in vendor['product']['product_data']:
                    product_name = product['product_name']
                    versions = [x['version_value'] for x in product['version']['version_data']]

                    new_df = pd.DataFrame([[CVE,vendor_name,product_name,versions]],
                                              columns=['CVE','vendor','product','version'])
                    df = df.append(new_df, ignore_index=True)           
        except IndexError:
            pass
    if save_loc:
        save_vuln_data(save_loc)
    return df


def save_vuln_data(df = None, save_loc = "data/vuln/data.pkl"):
    if not df:
        cve_data = read_vuln_json()
        df = parse_cve_json(cve_data)
    return df.to_pickle("data/vuln/data.pkl")


def load_vuln_data():
    try:
        return pd.read_pickle("data/vuln/data.pkl")
    except:
        save_vuln_data() #force a load/parse/save
        return pd.read_pickle("data/vuln/data.pkl")
        

In [3]:
#data_df = parse_cve_json(read_vuln_json())
#print(data_df.dtypes)

data = load_vuln_data()

In [5]:
vendors = data.vendor.sort_values().unique()
products = data['product'].sort_values().unique()

len(vendors)

1301

In [18]:
from neo4j.v1 import GraphDatabase

driver = GraphDatabase.driver("bolt://127.0.0.1:7687", auth=("neo4j", "Sn0wcrash"))


def drop_nodes(tx):
    tx.run("MATCH (n) DETACH DELETE n")

def add_node(tx, item, node_type):
    query_string = "MERGE (a:{} ".format(node_type)
    query_string+="{name: $name})"
    tx.run(query_string, name=item)
    #tx.run("MERGE (a:product {name: $name, vendor: $vend})",
     #      name=pdct, vend=vend)
    
def add_vendor(tx, pdct, vend):
    pass
    #tx.run("MERGE (a:product {name: $name, vendor: $vend})",
       #    name=pdct, vend=vend)
    
def add_version(tx, item, version, pdct):
    query_string = "MERGE (a:version {name: $name, version:$version, product: $pdct})"
    tx.run(query_string, name=item, version=version, pdct=pdct)
    #tx.run("MERGE (a:product {name: $name, vendor: $vend})",
          # name=pdct, vend=vend)

    
def add_vendor_relation(tx, pdct, vend):
    query_string = "MATCH (a:vendor {name: $value1}),"\
                    "(b:product {name: $value2})"\
                    "MERGE (a)-[r:MAKES]->(b)"
    tx.run(query_string,value1=vend, value2=pdct)
    
def add_version_relation(tx, version_name, version, pdct):
    query_string = "MATCH (a:version {name: $value1, version:$version}),"\
                    "(b:product {name: $value2})"\
                    "MERGE (a)-[r:VERSION_OF]->(b)"
    tx.run(query_string,value1=version_name, value2=pdct, version=version)
    
    
def add_cve_relation(tx, version_name, version, pdct, cve):
    query_string = "MATCH (a:CVE {name: $cve}),"\
                    "(b:version {name: $version_name, version:$version, product: $pdct})"\
                    "MERGE (a)-[r:AFFECTS]->(b)"
    #query_string= "MATCH (a:cve),(b:product)"\
    #               "WHERE a.name = $cve AND a.product = $product AND b.name = $version"\
    #                "CREATE (a)-[r:RELTYPE]->(b)"
    tx.run(query_string,cve=cve,version_name=version_name, version=version, pdct=pdct)
    #tx.run("MERGE (a:CVE {name: $name}) "
       #    "MERGE (a)-[:AFFECTS]->(version:Version {version: $version}) WITH version.product = $product",
       #    name=CVE, version=vsn, product=pdct)
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )


In [43]:
def populate_db():
    vendors = data.vendor.sort_values().unique()
    products = data['product'].sort_values().unique()
    print(mini_df)
    with driver.session() as session:
        session.write_transaction(drop_nodes)
        for vendor in log_progress(vendors):
            session.write_transaction(add_node, vendor, "vendor")
        for product in log_progress(products):
            session.write_transaction(add_node, product, "product")
        for k,item in log_progress(iter(data.iterrows()), every=1):

            session.write_transaction(add_vendor_relation, item["product"], item["vendor"])
            session.write_transaction(add_node, item["CVE"], "CVE")
            for version in item['version']:
                version_name = version+item["product"]
                session.write_transaction(add_version, version_name, version, item["product"])
                session.write_transaction(add_version_relation, version_name, version, item["product"])
                session.write_transaction(add_cve_relation, version_name, version, item["product"], 
                                          item["CVE"])
    
def add_exploit_node(tx, category, module, cve):
    query_string = "MERGE (a:exploit {name: $name, category:$category, cve:$cve})"
    tx.run(query_string, name=module, category=category, cve=cve)
    
def add_exploit_relation(tx, cve):
    query_string = "MATCH (a:CVE {name: $cve}),"\
                    "(b:exploit {cve: $cve})"\
                    "MERGE (b)-[r:EXPLOITS]->(a)"
    tx.run(query_string,cve=cve)
    
def load_db_exploit_nodes(exploits_json="data/exploit/data.json"):
    with open(exploits_json) as f:
        exploits=json.load(f)
    with driver.session() as session:
        for cve,data in exploits.items():
            for category,module in data.items():
                session.write_transaction(add_exploit_node, category, module, cve)
                
def load_db_exploit_relations(exploits_json="data/exploit/data.json"):  
    with open(exploits_json) as f:
        exploits=json.load(f)
    with driver.session() as session:
        for cve,data in exploits.items():
            session.write_transaction(add_exploit_relation, cve)

                

load_db_exploit_nodes()
load_db_exploit_relations()

In [39]:
products = data['product'].sort_values().unique()
len(products)

3090

In [45]:
for k,item in mini_df.iterrows():
    print(item['vendor'])

microsoft
microsoft
microsoft
microsoft
microsoft
microsoft
microsoft
microsoft
microsoft
microsoft


In [64]:
with driver.session() as session:
    for k,item in data.iterrows():
        for version in item['version']:
            session.write_transaction(add_cve_relation, version, item["product"], item["CVE"])
            

Exception ignored in: 'neo4j.bolt._io.ChunkedInputBuffer.receive'
Traceback (most recent call last):
  File "/home/khodges@corp.reliaquest.com/miniconda3/envs/wm/lib/python3.6/ssl.py", line 1009, in recv_into
    return self.read(nbytes, buffer)
  File "/home/khodges@corp.reliaquest.com/miniconda3/envs/wm/lib/python3.6/ssl.py", line 871, in read
    return self._sslobj.read(len, buffer)
  File "/home/khodges@corp.reliaquest.com/miniconda3/envs/wm/lib/python3.6/ssl.py", line 631, in read
    v = self._sslobj.read(len, buffer)
KeyboardInterrupt: 


ServiceUnavailable: Failed to write to closed connection Address(host='127.0.0.1', port=7687)